In [19]:
import pandas as pd
import numpy as np

pd.options.display.float_format = '{:.2f}%'.format

**Columns not useful as features**: 
- **encounter_id**: Unique, non-repeating values. 
- **hospital_id-icu_id**: A patient diagnosed with Diabetus Mellitus should not depend on hospital_id & it's icu_id. Each hostpital will have unique icu_ids. DROP
- **readmission_status**: is 0 for all rows. DROP
- **labs & vitals category**: They have max and min values. Should these be replaced with average?
- **bmi**: Should weight and height be replaced only with bmi?
- **height, weight**: Fill the missing values with the mean values according to the gender.

In [3]:
df = pd.read_csv("data/TrainingWiDS2021.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130157 entries, 0 to 130156
Columns: 181 entries, Unnamed: 0 to diabetes_mellitus
dtypes: float64(157), int64(18), object(6)
memory usage: 179.7+ MB


In [4]:
df.describe() #[['h1_temp_max', 'h1_temp_min']]

,Unnamed: 0,encounter_id,hospital_id,age,bmi,elective_surgery,height,icu_id,pre_icu_los_days,readmission_status,...,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
count,130157.000000,130157.000000,130157.000000,125169.000000,125667.000000,130157.000000,128080.000000,130157.000000,130157.000000,130157.0,...,16760.000000,16760.000000,130157.00000,130157.000000,130157.000000,130157.000000,130157.000000,130157.000000,130157.000000,130157.000000
mean,65079.000000,213000.856519,106.102131,61.995103,29.110260,0.189840,169.607219,662.428344,0.839933,0.0,...,247.525419,239.617358,0.00103,0.016081,0.013599,0.025669,0.007307,0.004187,0.020852,0.216285
std,37573.233831,38109.828146,63.482277,16.822880,8.262776,0.392176,10.833085,304.259843,2.485337,0.0,...,131.440167,128.562211,0.03207,0.125786,0.115819,0.158146,0.085166,0.064574,0.142888,0.411712
min,1.000000,147000.000000,1.000000,0.000000,14.844926,0.000000,137.200000,82.000000,-0.250000,0.0,...,42.000000,38.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,32540.000000,180001.000000,49.000000,52.000000,23.598006,0.000000,162.500000,427.000000,0.045833,0.0,...,144.000000,138.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,65079.000000,213014.000000,112.000000,64.000000,27.564749,0.000000,170.100000,653.000000,0.155556,0.0,...,228.125000,218.750000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,97618.000000,246002.000000,165.000000,75.000000,32.803127,0.000000,177.800000,969.000000,0.423611,0.0,...,333.000000,324.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,130157.000000,279000.000000,204.000000,89.000000,67.814990,1.000000,195.590000,1111.000000,175.627778,0.0,...,720.000000,654.813793,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [5]:
df_test = pd.read_csv("data/UnlabeledWiDS2021.csv")
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10234 entries, 0 to 10233
Columns: 180 entries, Unnamed: 0 to solid_tumor_with_metastasis
dtypes: float64(156), int64(18), object(6)
memory usage: 14.1+ MB


In [6]:
df.groupby(['hospital_id']).count()['encounter_id'].sort_values()

hospital_id
25        2
130       2
93        6
95        6
23        7
       ... 
7      2944
86     2962
188    3075
19     3885
118    4306
Name: encounter_id, Length: 204, dtype: int64

In [7]:
df.loc[df['hospital_id']==4]

,Unnamed: 0,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,...,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
8393,8394,173617,4,56.0,23.724537,0,Caucasian,M,177.8,Emergency Department,...,NaN,NaN,0,0,0,0,0,0,0,0
8570,8571,188717,4,NaN,29.257812,0,NaN,F,160.0,Operating Room,...,NaN,NaN,0,0,0,0,0,0,0,0
8634,8635,276136,4,26.0,28.666947,0,Other/Unknown,F,172.7,Emergency Department,...,666.666667,654.813793,0,0,0,0,0,0,0,0
9522,9523,251919,4,NaN,19.547261,0,Caucasian,F,152.4,Emergency Department,...,NaN,NaN,0,0,0,0,0,0,0,0
9638,9639,197182,4,74.0,47.949208,0,Caucasian,F,165.1,Floor,...,NaN,NaN,0,0,0,0,0,0,0,0
9723,9724,275265,4,NaN,26.191889,0,Caucasian,M,177.8,Floor,...,NaN,NaN,0,0,0,0,0,0,0,0
9859,9860,177497,4,65.0,30.792556,0,Caucasian,F,170.2,Floor,...,NaN,NaN,0,0,0,0,0,0,0,0


In [8]:
df.groupby('icu_id')['icu_id'].agg(['count']).sort_values(by = 'count', ascending=False)

,count
icu_id,
1019,1344
646,1312
653,1296
876,1280
413,1219
...,...
365,2
241,2
989,2


In [9]:
df['bmi_calc'] = df['weight']/(df['height']/100)**2

In [10]:
df_bmi = df[['weight', 'height', 'bmi', 'bmi_calc']]
df_bmi_1  = df_bmi[df_bmi.isna().any(axis=1)]

In [11]:
df_bmi_1

,weight,height,bmi,bmi_calc
4,NaN,188.0,NaN,NaN
7,NaN,165.0,NaN,NaN
8,NaN,170.2,NaN,NaN
12,NaN,175.3,NaN,NaN
13,NaN,172.7,NaN,NaN
...,...,...,...,...
129554,NaN,180.3,NaN,NaN
129561,99.8,NaN,NaN,NaN
129901,NaN,160.0,NaN,NaN
130140,NaN,NaN,NaN,NaN


BMI calculations - relation to diabetes
bmi = weight (kg) / [height (m)]2

Adults:
For adults, the interpretation of BMI does not depend on sex or age. 
For adults 20 years old and older, BMI is interpreted using standard weight status categories.
BMI	Weight Status
Below 18.5	Underweight
18.5 – 24.9	Normal or Healthy Weight
25.0 – 29.9	Overweight
30.0 and Above	Obese


In [ ]:
df_bmi_1[ df_bmi_1['height'].isnull() & df_bmi_1['weight'].notnull() |
          df_bmi_1['height'].notnull() & df_bmi_1['weight'].isnull() |
          df_bmi_1['height'].isnull() & df_bmi_1['weight'].isnull()].shape

In [ ]:
df_bmi_1[ df_bmi_1['bmi'].isnull()].shape

In [ ]:
# Wherever either weight or height or both are not available, bmi is also not available.
# Isn't bmi dependent on gender
# There are no rows where height and weight are null and bmi is not null
'''print("Height-Nan:", df_bmi_1[df_bmi_1['height'].isnull().shape))
print("Weight-Nan:", df_bmi_1[df_bmi_1['height'].isnull().shape))
print("Height-Nan:", df_bmi_1[df_bmi_1['height'].isnull().shape))'''
print(df_bmi_1[df_bmi_1['height'].isnull() & df_bmi_1['weight'].isnull() & df_bmi_1['bmi'].notnull()])
print(df_bmi_1[df_bmi_1['height'].notnull() & df_bmi_1['weight'].isnull() & df_bmi_1['bmi'].notnull()])
print(df_bmi_1[df_bmi_1['height'].isnull() & df_bmi_1['weight'].notnull() & df_bmi_1['bmi'].notnull()])
print(df_bmi_1[df_bmi_1['height'].notnull() & df_bmi_1['weight'].notnull() & df_bmi_1['bmi'].isnull()])

In [ ]:
bins = pd.cut(df['bmi_calc'], [0.0, 18.5, 24.9, 30, 70], labels=['uw','nw','ow','o'])
df.groupby(bins)['bmi_calc'].agg(['count'])

In [ ]:
# underweight, normal, overweight, obese
bmi_bins = [0, 18.5, 24.9, 30, 70] 
bmi_labels=[    '[0.0-18.5] underweight',
                            '[18.5-24.9] normal weight',
                            '[30-70] over weight',
                            '[70<] obese']


In [ ]:
bins = pd.cut(df['bmi'], bmi_bins, labels=bmi_labels)
df_bmi = df.groupby(bins)['bmi'].agg(['count'])
df_bmi = df_bmi.reset_index()
df_bmi

In [ ]:
df_bmi_F = df[df['gender']=='F']
bins = pd.cut(df_bmi_F['bmi'], bmi_bins, labels=bmi_labels)
df_bmi_F = df_bmi_F.groupby(bins)['bmi'].agg(['count'])
df_bmi_F = df_bmi_F.reset_index()
df_bmi_F = df_bmi_F.rename(columns = {'count':'Female'})
df_bmi_F

In [ ]:
df_bmi = df_bmi_M.join(df_bmi_F.set_index('bmi'), on= 'bmi')
df_bmi['total'] = df_bmi['Male'] + df_bmi['Female']
df_bmi['%'] = 100 * df_bmi['total'] / df_bmi['total'].sum()
df_bmi

In [ ]:
table = pd.pivot_table(df, values='encounter_id', index='gender', columns='diabetes_mellitus', aggfunc='count')
table

In [ ]:
df_1 = df[df['diabetes_mellitus']==1]
bins = pd.cut(df_1['bmi'], bmi_bins, labels=bmi_labels)
df_bmi_1 = df_1.groupby(bins)['bmi'].agg(['count'])
df_bmi_1 = df_bmi_1.reset_index()
df_bmi_1 = df_bmi_1.rename(columns = {'count':'diabetes=1'})

In [ ]:
df_0 = df[df['diabetes_mellitus']==0]
bins = pd.cut(df_0['bmi'], bmi_bins, labels=bmi_labels)
df_bmi_0 = df_0.groupby(bins)['bmi'].agg(['count'])
df_bmi_0 = df_bmi_0.reset_index()
df_bmi_0 = df_bmi_0.rename(columns = {'count':'diabetes=0'})

In [ ]:
df_bmi = df_bmi_1.join(df_bmi_0.set_index('bmi'), on= 'bmi')
df_bmi['total'] = df_bmi['diabetes=1'] + df_bmi['diabetes=0']
df_bmi['%'] = 100 * df_bmi['total'] / df_bmi['total'].sum()
df_bmi

In [ ]:
df_bmi_M = df[df['gender']=='M']
bins = pd.cut(df_bmi_M['bmi'], bmi_bins, labels=bmi_labels)
df_bmi_M = df_bmi_M.groupby(bins)['bmi'].agg(['count'])
df_bmi_M = df_bmi_M.reset_index()
df_bmi_M = df_bmi_M.rename(columns = {'count':'Male'})
df_bmi_M

In [ ]:
print(df['age'].min(), df['age'].max())

In [ ]:
df[df['age']==0.0]['age'].count()

In [ ]:
pd.options.display.float_format = '{:.2f}%'.format

In [ ]:
age_bins = [-0.1, 0, 13, 20, 40, 60, 90]

In [ ]:
# Convert bmi from numeric to categorical - age-groups?
df_1 = df[df['diabetes_mellitus']==1]
bins = pd.cut(df_1['age'], age_bins)
df_age_1 = df_1.groupby(bins)['age'].agg(['count'])
df_age_1 = df_age_1.reset_index()
df_age_1 = df_age_1.rename(columns = {'count':'diabetes=1'})

In [ ]:
# Convert bmi from numeric to categorical - age-groups?
df_0 = df[df['diabetes_mellitus']==0]
bins = pd.cut(df_0['age'], age_bins)
df_age_0 = df_0.groupby(bins)['age'].agg(['count'])
df_age_0 = df_age_0.reset_index()
c

In [ ]:
df_age = df_age_1.join(df_age_0.set_index('age'), on= 'age')
df_age['total'] = df_age['diabetes=1'] + df_age['diabetes=0']
df_age['%'] = 100 * df_age['total'] / df_age['total'].sum()
df_age

**Conclusions**
- Remove the rows with age = 0
- Categorize bmi into 4 categories


In [23]:
 pd.pivot_table(df, values='age', index=['age'],
                    columns=['diabetes_mellitus'], aggfunc=np.sum, fill_value=0)

ValueError: Grouper for 'age' not 1-dimensional

In [12]:
# encounter_ids are unique and there are no repeats
df_temp = df.groupby(['encounter_id'])['encounter_id'].count().sort_values(ascending=False)
#df_temp = df_temp.rename(columns = {'encounter_id':'count'})
#df_temp['%'] = 100 * df_temp['count'] / df_temp['count'].sum()
#pd.options.display.float_format = '{:.2f}%'.format
print(df_temp)

encounter_id
147000    1
235009    1
235022    1
235021    1
235020    1
         ..
191006    1
191005    1
191004    1
191003    1
279000    1
Name: encounter_id, Length: 130157, dtype: int64


In [20]:
def getAllValues(df, col):
    df_temp = df.groupby([col]).count()['encounter_id'].sort_values(ascending=False).reset_index()
    df_temp = df_temp.rename(columns = {'encounter_id':'count'})
    df_temp['%'] = 100 * df_temp['count'] / df_temp['count'].sum()
    print(df_temp)

In [18]:
# Columns - string type
for  col in ['ethnicity', 'gender', 'hospital_admit_source', 'icu_admit_source', 'icu_type', 'icu_stay_type']:
    getAllValues(df, col)
    getAllValues(df_test, col)
    print()

ethnicity   count      %
0         Caucasian  100236 77.96%
1  African American   13911 10.82%
2     Other/Unknown    6261  4.87%
3          Hispanic    5049  3.93%
4             Asian    2198  1.71%
5   Native American     915  0.71%
          ethnicity  count      %
0         Caucasian   7939 79.15%
1  African American    931  9.28%
2          Hispanic    471  4.70%
3     Other/Unknown    435  4.34%
4             Asian    171  1.70%
5   Native American     83  0.83%

  gender  count      %
0      M  70518 54.21%
1      F  59573 45.79%
  gender  count      %
0      M   5525 54.01%
1      F   4704 45.99%

   hospital_admit_source  count      %
0   Emergency Department  51258 52.87%
1         Operating Room  13757 14.19%
2                  Floor  11324 11.68%
3           Direct Admit   8253  8.51%
4          Recovery Room   4563  4.71%
5         Other Hospital   2417  2.49%
6       Acute Care/Floor   2097  2.16%
7   Step-Down Unit (SDU)   1607  1.66%
8                   PACU   1031  1.0

In [22]:
# Columns - binary type
for col in ['elective_surgery', 'readmission_status', 'diabetes_mellitus' ]:
    getAllValues(df, col)	
    if col != 'diabetes_mellitus':
        getAllValues(df_test, col)
    print()

elective_surgery   count      %
0                 0  105448 81.02%
1                 1   24709 18.98%
   elective_surgery  count      %
0                 0   8186 79.99%
1                 1   2048 20.01%

   readmission_status   count       %
0                   0  130157 100.00%
   readmission_status  count       %
0                   0  10234 100.00%

   diabetes_mellitus   count      %
0                  0  102006 78.37%
1                  1   28151 21.63%



In [ ]:
# Columns - nan data

nan_counts = []
nan_counts_percent = []
for col in df.columns:
    total = df[col].count()
    nan_rows = len(df[df[col].isnull()])
    nan_counts.append(nan_rows)
    nan_counts_percent.append(100 * nan_rows/(total+nan_rows))

In [ ]:
df_nan = pd.DataFrame({'Variable Name':df.columns, 'nan_counts':nan_counts, '%':nan_counts_percent})
df_nan = df_nan.sort_values('nan_counts', ascending = False)
df_nan['%'] = df_nan['%'].map('{:,.1f}%'.format)
print(df_nan)
df_nan.to_csv("nan_data.csv")
print(df_nan.shape[0])

In [ ]:
# Rows with nan data
df_nan = df[df.isna().any(axis=1)]
print (f"Rows with nan values: {100 * df_nan.shape[0]/df.shape[0]:.2f}%")

In [ ]:
df_dict = pd.read_csv("data/DataDictionaryWiDS2021.csv")
print(df_dict[['Variable Name', 'Category', 'Data Type']])
print(df_dict.shape[0])

In [ ]:
df_data_desc = pd.merge(df_nan, df_dict, on="Variable Name")
df_data_desc = df_data_desc.sort_values(by=['Category','Variable Name'])
df_data_desc.to_csv("datadesc.csv")

In [ ]:
df[['apache_2_diagnosis', 'diabetes_mellitus']].corr()

In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np
X = df[['arf_apache']]
Y = df['diabetes_mellitus']
slm = LinearRegression()
np.set_printoptions(precision=2)
slm.fit(X,Y)

In [ ]:
Yhat =slm_hm.predict(X_hm)